In [1]:
#import

import pandas as pd
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
# Create pathway to mock data
path = Path("Crimes_-_2001_to_Present.csv")

In [3]:
# load dataframe
crime_df = pd.read_csv(path)
crime_df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,12.0,61.0,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,29.0,25.0,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,...,8.0,44.0,06,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN
3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,...,35.0,21.0,18,1152037.0,1920384.0,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)"
4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,28.0,25.0,08A,1141706.0,1900086.0,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)"


In [4]:
crime_df = crime_df[crime_df["Year"] >= 2010]

In [5]:
# Taking Count
crime_df.count()

ID                      3590612
Case Number             3590610
Date                    3590612
Block                   3590612
IUCR                    3590612
Primary Type            3590612
Description             3590612
Location Description    3581058
Arrest                  3590612
Domestic                3590612
Beat                    3590612
District                3590611
Ward                    3590516
Community Area          3590215
FBI Code                3590612
X Coordinate            3555277
Y Coordinate            3555277
Year                    3590612
Updated On              3590612
Latitude                3555277
Longitude               3555277
Location                3555277
dtype: int64

In [6]:
# Drop na values
crime_df = crime_df.dropna()

In [7]:
# Validate drops in na
crime_df.count()

ID                      3548595
Case Number             3548595
Date                    3548595
Block                   3548595
IUCR                    3548595
Primary Type            3548595
Description             3548595
Location Description    3548595
Arrest                  3548595
Domestic                3548595
Beat                    3548595
District                3548595
Ward                    3548595
Community Area          3548595
FBI Code                3548595
X Coordinate            3548595
Y Coordinate            3548595
Year                    3548595
Updated On              3548595
Latitude                3548595
Longitude               3548595
Location                3548595
dtype: int64

In [8]:
# dropping unnessary columns ( atbitrary placeholders)
cleaned_crime_df = crime_df.drop(columns = ["Case Number", "ID", "IUCR", "Block", "Year", "Updated On", "Date", "Description", "FBI Code", "Location"])

In [9]:
#Adding labelencoder for Primaty Type of crime
le = LabelEncoder()
cleaned_crime_df['Primary Type'] = le.fit_transform(cleaned_crime_df['Primary Type'])
cleaned_crime_df['Location Description'] = le.fit_transform(cleaned_crime_df['Location Description'])

In [10]:
cleaned_crime_df.head(10)

,Primary Type,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,X Coordinate,Y Coordinate,Latitude,Longitude
0,2,147,False,True,924,9.0,12.0,61.0,1165074.0,1875917.0,41.815117,-87.670000
1,33,64,False,False,1511,15.0,29.0,25.0,1138875.0,1904869.0,41.895080,-87.765400
3,18,170,True,False,1412,14.0,35.0,21.0,1152037.0,1920384.0,41.937406,-87.716650
4,1,17,False,True,1522,15.0,28.0,25.0,1141706.0,1900086.0,41.881903,-87.755121
5,3,147,False,False,614,6.0,21.0,71.0,1168430.0,1850165.0,41.744379,-87.658431
6,3,152,False,False,1434,14.0,32.0,24.0,1161628.0,1912157.0,41.914636,-87.681631
7,33,97,True,False,1034,10.0,25.0,31.0,1159734.0,1889313.0,41.851989,-87.689219
8,30,175,False,True,1222,12.0,27.0,27.0,1155536.0,1900515.0,41.882814,-87.704326
10,33,136,False,False,833,8.0,13.0,65.0,1150938.0,1857056.0,41.763648,-87.722345
11,2,171,False,False,1623,16.0,45.0,11.0,1137969.0,1934340.0,41.975968,-87.768014


In [11]:
# Define the features set.
X = cleaned_crime_df.copy()
X = X.drop("Arrest", axis=1)
X.head(10)

,Primary Type,Location Description,Domestic,Beat,District,Ward,Community Area,X Coordinate,Y Coordinate,Latitude,Longitude
0,2,147,True,924,9.0,12.0,61.0,1165074.0,1875917.0,41.815117,-87.670000
1,33,64,False,1511,15.0,29.0,25.0,1138875.0,1904869.0,41.895080,-87.765400
3,18,170,False,1412,14.0,35.0,21.0,1152037.0,1920384.0,41.937406,-87.716650
4,1,17,True,1522,15.0,28.0,25.0,1141706.0,1900086.0,41.881903,-87.755121
5,3,147,False,614,6.0,21.0,71.0,1168430.0,1850165.0,41.744379,-87.658431
6,3,152,False,1434,14.0,32.0,24.0,1161628.0,1912157.0,41.914636,-87.681631
7,33,97,False,1034,10.0,25.0,31.0,1159734.0,1889313.0,41.851989,-87.689219
8,30,175,True,1222,12.0,27.0,27.0,1155536.0,1900515.0,41.882814,-87.704326
10,33,136,False,833,8.0,13.0,65.0,1150938.0,1857056.0,41.763648,-87.722345
11,2,171,False,1623,16.0,45.0,11.0,1137969.0,1934340.0,41.975968,-87.768014


In [12]:
# Define the target set.
y = cleaned_crime_df["Arrest"].ravel()
y[:5]

array([False, False,  True, False, False])

In [13]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size=0.25)

In [14]:
# Creating a StandardScaler
scaler = StandardScaler()

# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
#WARNING: COMPUTATIONALLY HEAVY
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=56, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [16]:
# Making predictions using X_test_scaled
predictions = rf_model.predict(X_test_scaled)
predictions

array([False, False,  True, ..., False, False, False])

In [17]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,1940361,109708
Actual 1,241880,369498
